In [0]:
!mkdir train_images

import cv2
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

def img_compute(img):
	
	#img = cv2.resize(img, (128,128), 3)
	#print(img.shape)
	#img = np.reshape(img, (128, 128), 1)
	img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


	gray = cv2.threshold(img, 0, 225, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

	#kernel = np.ones((2,2),np.uint8)
	#gray = cv2.erode(gray,kernel,iterations = 1)

	return gray

n=0

for img_file in glob.glob('./AI4Good---Meza-OCR-Challenge/cell_images/training_set/*.jpg'):
  im = cv2.imread(img_file)
  im_n = img_compute(im)
	#im_n = im_n.flatten()
  n = n+1
  file_n = img_file
  file_n = file_n.split("training_set/", 1)
  file_n = file_n[1]
  #print('working here ', file_n)
  cv2.imwrite(os.path.join('./train_images/', file_n), im_n)
  


In [0]:
!rm -r utils.py

In [0]:
#! git clone https://github.com/Charitable-Analytics-International/AI4Good---Meza-OCR-Challenge.git
#!sudo apt install tesseract-ocr
#!pip install pytesseract

In [0]:
import numpy as np
import os
import shutil
import tensorflow as tf
import cv2
import datetime
import logging
import time

In [0]:
DATADIR = '/content/AI4Good---Meza-OCR-Challenge/cell_images/training_set_values.txt'
VALID_DIR = '/content/AI4Good---Meza-OCR-Challenge/cell_images/validation_set_values.txt'

In [0]:
import numpy as np
def load_labels(file):
    labels = list(open(file).readlines())
    labels = [s.strip() for s in labels[1:]]
    labels = [s.split(';') for s in labels]

    labels_dict = dict(labels)

    labels = np.asarray(labels, dtype=str)
    labels = labels[:, 1]

    return labels, labels_dict

In [0]:
l, ld = load_labels(DATADIR)

In [0]:
def process(k,img):
        thresh = cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU
        low = 80
        proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_BGR2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
        data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 6').replace('Rina','Ring')
        return data


In [0]:
def load_img_path(images_path):
    tmp = os.listdir(images_path)
    tmp.sort(key=lambda x: int(x.split('.')[0]))

    file_names = [images_path + s for s in tmp]

    file_names = np.asarray(file_names)

    return file_names

In [0]:
file_names = load_img_path('./train_images/')
#valid_names = load_img_path('/content/AI4Good---Meza-OCR-Challenge/cell_images/validation_set/')

In [80]:
file_names

array(['./train_images/1.jpg', './train_images/2.jpg',
       './train_images/3.jpg', ..., './train_images/7495.jpg',
       './train_images/7496.jpg', './train_images/7497.jpg'], dtype='<U23')

In [0]:
def split_train_val(X, y, train_size):
    """Split dataset for training and validation.

    Args:
        X: A 1-D numpy array containing pathes of images.
        y: A 1-D numpy array containing labels.
        train_size: Size of training data to split.
    Returns:
        1-D numpy array having the same definition with X and y.
    """

    total_size = len(X)
    # shuffle data
    shuffle_indices = np.random.permutation(np.arange(total_size))
    X = X[shuffle_indices]
    y = y[shuffle_indices]

    # split training data
    train_indices = np.random.choice(total_size, train_size, replace=False)
    X_train = X[train_indices]
    y_train = y[train_indices]

    # split validation data
    val_indices = [i for i in range(total_size) if i not in train_indices]
    X_val = X[val_indices]
    y_val = y[val_indices]

    return X_train, y_train, X_val, y_val

In [0]:
file_names, l, valid_names, v = split_train_val(file_names, l, 6000)

In [0]:
def write_to_file(data, file_to_output='./X_train.txt'):
    """Write X_train/y_train/X_val/y_val/X_infer to file for further
       processing (e.g. make input queue of tensorflow).
    Args:
        data: A 1-D numpy array, e.g, X_train/y_train/X_val/y_val/X_infer.
        file_to_output: A file to store data.
    """
    # with open('X_train.csv','a') as f_handle:
    #     np.savetxt(f_handle, X_train, fmt='%s', delimiter=",")

    with open(file_to_output, 'w') as f:
        for item in data.tolist():
            f.write(item + '\n')

In [0]:
write_to_file(file_names)
write_to_file(valid_names, file_to_output='./X_val.txt')

In [0]:
write_to_file(l, file_to_output='./y_train.txt')
write_to_file(v, file_to_output='./y_val.txt')

In [0]:
def cp_file(imgs_list_para, labels_list_para, dst_para):
    for i in range(imgs_list_para.shape[0]):
        file_path = imgs_list_para[i]

        filename = os.path.basename(file_path)
        fn = filename.split('.')[0]
        ext = filename.split('.')[1]

        dest_filename = dst_para + fn + '_' + labels_list_para[i] + '.' + ext


        shutil.copyfile(file_path, dest_filename)

In [0]:
!rm -rf train

In [0]:
!rm -rf val

In [0]:
!mkdir train
!mkdir val

In [0]:
cp_file(file_names, l, './train/')
cp_file(valid_names, v, './val/')

In [0]:
def load_data(file_to_read):
    """Load X_train/y_train/X_val/y_val/X_infer for further
       processing (e.g. make input queue of tensorflow).
    Args:
        file_to_read:
    Returns:
        X_train/y_train/X_val/y_val/X_infer.
    """

    data = np.recfromtxt(file_to_read)
    data = np.asarray(data)

    return data

In [0]:
!rm -rf checkpoint

In [0]:
!rm -r utils.py

In [0]:
!python ./main.py --train_dir=./train/ \
  --val_dir=./val/ \
  --image_height=128 \
  --image_width=128 \
  --image_channel=1 \
  --out_channels=64 \
  --num_hidden=128 \
  --batch_size=128 \
  --log_dir=./log/train \
  --num_gpus=1 \
  --mode=train

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.


feature_h: 8, feature_w: 8
lstm input shape: [128, 8, 512]
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
loading train data
size:  6000
loading validation data
size: 1497

2019-06-09 14:59:33.515366: I tensorflow/

In [0]:
!python ./main.py --infer_dir=/content/AI4Good---Meza-OCR-Challenge/cell_images/validation_set/ \
  --checkpoint_dir=./checkpoint/ \
  --num_gpus=0 \
  --batch_size=29 \
  --mode=infer